In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

import json
import pickle

In [2]:
df = pd.read_csv("bgg_100_10.csv")


#df must have the columns ID______user________rating
id_c = CategoricalDtype(sorted(df.ID.unique()), ordered=True)
user_c = CategoricalDtype(sorted(df.user.unique()), ordered=True)
row = df.user.astype(user_c).cat.codes
col = df.ID.astype(id_c).cat.codes
sparse_matrix = csr_matrix((df["rating"], (row, col)), \
                           shape=(user_c.categories.size, id_c.categories.size))


In [3]:
user_c.categories

Index([' mycroft', '-=Yod@=-', '-Johnny-', '-Loren-', '-LucaS-', '-mIDE-',
       '-mik-', '-pj-', '-snarf-', '-toni-',
       ...
       'zzz2525', 'zzzabiss', 'zzzeagle', 'zzzk1', 'zzzkardel', 'zzzuzu',
       'zzzvone', 'zzzxxxyyy', 'zzzzzane', 'Æleksandr Þræð'],
      dtype='object', length=192092)

In [4]:
id_c.categories

Int64Index([     1,      2,      3,      4,      5,      7,      8,      9,
                10,     11,
            ...
            299252, 301919, 302193, 302260, 302336, 302388, 302723, 303051,
            303057, 305682],
           dtype='int64', length=10617)

There are two dictionaries:
dict_cat_to_ID sends categorical data(row number of a sparse matrix) to game ID's
dict_ID_to_cat sends the game ID to the categorical data(row number of a sparse matrix)

In [143]:
dict_cat_to_ID = dict(zip(range(len(list(id_c.categories))), list(id_c.categories)))

In [27]:
dict_cat_to_ID["5651"]

100423

In [145]:
dict_ID_to_cat = dict(zip(list(id_c.categories), range(len(list(id_c.categories)))))

In [146]:
dict_ID_to_cat[100423]

5651

In [157]:
with open('dict_cat_to_ID.txt', 'w') as file:
    json.dump(dict_cat_to_ID, file)


with open('dict_ID_to_cat.txt', 'w') as file:
    json.dump(dict_ID_to_cat, file)

Let's build the knn model and pickle it as well:

user-by-item matrix

In [5]:
sparse_matrix.shape

(192092, 10617)

In [6]:
from svd_with_bias import *

In [7]:
U,S,V  = svd_bias(sparse_matrix, 100)

In [8]:
V.shape

(100, 10617)

rows as games:

In [9]:
item_matrix = csr_matrix(np.dot(V.T, np.diag(S)))

In [10]:
item_matrix

<10617x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1061700 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(item_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
model_knn

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
knnPickle = open('knnpickle_file', 'wb') 
pickle.dump(model_knn, knnPickle)  


Loading the data back:

In [2]:
with open('dict_cat_to_ID.txt', 'r') as file:
    dict_cat_to_ID = json.load(file)

In [3]:
with open('dict_ID_to_cat.txt', 'r') as file:
    dict_ID_to_cat = json.load(file)

In [4]:
with open('knnpickle_file', 'rb') as file:
    loaded_model = pickle.load(file)

In [5]:
graph = loaded_model.kneighbors_graph(n_neighbors=100)

In [6]:
game_dict = {c:[dict_cat_to_ID.get(str(x)) for x in graph[dict_ID_to_cat.get(c)].indices] \
             for c in dict_ID_to_cat.keys()}

In [7]:
dict_ID_to_cat.get("100423")

5651

In [8]:
print(game_dict.get("269207"))

[170624, 229265, 163839, 244711, 182134, 281442, 190082, 227789, 255692, 245934, 234277, 187700, 232405, 196526, 165722, 283863, 119506, 109125, 204286, 164949, 236191, 69779, 210290, 247367, 276498, 234671, 197405, 193670, 192458, 157917, 160495, 214887, 225244, 252526, 272682, 276042, 302336, 119637, 262733, 245422, 166317, 270844, 181495, 261424, 280794, 208428, 171726, 160010, 103670, 139952, 174802, 235014, 240980, 36597, 68199, 57998, 183572, 125977, 131904, 298572, 18748, 275067, 10679, 217083, 25900, 31759, 17394, 254888, 101929, 56995, 12898, 241987, 156175, 250488, 241225, 217372, 2251, 185123, 113931, 293678, 216179, 7349, 122240, 12355, 302388, 2081, 267127, 252399, 262543, 11967, 13016, 254640, 203800, 6431, 87907, 229491, 232924, 149951, 19622, 91080]


In [9]:
with open("game_dict.txt", "w") as file:
    json.dump(game_dict, file)

In [12]:
len(game_dict["1"])

100

IT WORKS!!!!!!!